## Using question answering pipeline

In [1]:
from transformers import pipeline
nlp = pipeline("question-answering")
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

In [2]:
result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
result = nlp(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question.', score: 0.6226, start: 34, end: 96
Answer: 'SQuAD dataset,', score: 0.5053, start: 147, end: 161


## Using a model and a tokenize

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad", return_dict=True)

In [5]:
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

In [6]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print("text_tokens: ", text_tokens)
    
    #answer_start_scores, answer_end_scores = model(**inputs)
    outputs = model(**inputs)
    answer_start_scores = outputs[0]
    print("answer_start_scores: ", answer_start_scores)
    
    answer_end_scores = outputs[1]
    print("answer_end_scores: ", answer_end_scores)
    
    answer_start = torch.argmax(answer_start_scores)  # Get the most likely beginning of answer with the argmax of the score
    print("start_max: ", torch.max(answer_start_scores), " start_argmax: ", torch.argmax(answer_start_scores))
    
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    print("end_max: ", torch.max(answer_end_scores), " end_argmax: ", torch.argmax(answer_end_scores))
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    
    print(f"Question: {question}")
    print(f"Answer({answer_start},{answer_end}): {answer}")

text_tokens:  ['[CLS]', 'how', 'many', 'pre', '##train', '##ed', 'models', 'are', 'available', 'in', '[UNK]', 'transformers', '?', '[SEP]', '[UNK]', 'transformers', '(', 'formerly', 'known', 'as', 'p', '##yt', '##or', '##ch', '-', 'transformers', 'and', 'p', '##yt', '##or', '##ch', '-', 'pre', '##train', '##ed', '-', 'bert', ')', 'provides', 'general', '-', 'purpose', 'architecture', '##s', '(', 'bert', ',', 'gp', '##t', '-', '2', ',', 'roberta', ',', 'xl', '##m', ',', 'di', '##sti', '##lbert', ',', 'xl', '##net', '…', ')', 'for', 'natural', 'language', 'understanding', '(', 'nl', '##u', ')', 'and', 'natural', 'language', 'generation', '(', 'nl', '##g', ')', 'with', 'over', '32', '+', 'pre', '##train', '##ed', 'models', 'in', '100', '+', 'languages', 'and', 'deep', 'inter', '##oper', '##ability', 'between', 'tensor', '##flow', '2', '.', '0', 'and', 'p', '##yt', '##or', '##ch', '.', '[SEP]']
answer_start_scores:  tensor([[-6.5990, -6.1387, -7.8761, -7.8292, -8.7167, -8.9216, -8.6420, -8